In [1]:
import sys, os, time
from pathlib import Path
from glob import glob

import numpy as np
import matplotlib.pyplot as plt

import scipy.io


def humansize(nbytes):
    '''From https://stackoverflow.com/questions/14996453/python-libraries-to-calculate-human-readable-filesize-from-bytes'''
    suffixes = ['B', 'KB', 'MB', 'GB', 'TB', 'PB']
    i = 0
    while nbytes >= 1024 and i < len(suffixes)-1:
        nbytes /= 1024.
        i += 1
    f = ('%.2f' % nbytes).rstrip('0').rstrip('.')
    return '%s %s' % (f, suffixes[i])

ds_path = '/mnt/exdisk1/danigil/datasets/imagenet'

path_imagenet_val_dataset = Path(os.path.join(ds_path, "processed")) # path/to/data/
dir_images = Path(os.path.join(ds_path, "ILSVRC2012_img_val")) # path/to/images/directory
path_labels = Path(os.path.join(ds_path, "ILSVRC2012_validation_ground_truth.txt"))
path_synset_words = Path(os.path.join(ds_path, "synset_words.txt"))
path_meta = Path(os.path.join(ds_path, "meta.mat"))

image_paths = sorted(glob(str(dir_images/"*")))
n_images = len(image_paths)

def create_imagenet_preprocessed_y():
    meta = scipy.io.loadmat(str(path_meta))
    original_idx_to_synset = {}
    synset_to_name = {}

    for i in range(1000):
        ilsvrc2012_id = int(meta["synsets"][i,0][0][0][0])
        synset = meta["synsets"][i,0][1][0]
        name = meta["synsets"][i,0][2][0]
        original_idx_to_synset[ilsvrc2012_id] = synset
        synset_to_name[synset] = name

    synset_to_keras_idx = {}
    keras_idx_to_name = {}
    with open(str(path_synset_words), "r") as f:
        for idx, line in enumerate(f):
            parts = line.split(" ")
            synset_to_keras_idx[parts[0]] = idx
            keras_idx_to_name[idx] = " ".join(parts[1:])

    convert_original_idx_to_keras_idx = lambda idx: synset_to_keras_idx[original_idx_to_synset[idx]]

    with open(str(path_labels),"r") as f:
        y_val = f.read().strip().split("\n")
        y_val = np.array([convert_original_idx_to_keras_idx(int(idx)) for idx in y_val])

    return y_val

In [2]:
y_val = create_imagenet_preprocessed_y()

In [4]:
y_val.shape

(50000,)

In [5]:
n_images

50000

In [8]:
import tensorflow as tf

ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=dir_images,
    labels=list(y_val),
    label_mode='int',

    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(256, 256),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 50000 files belonging to 1000 classes.


2024-08-16 17:18:06.817950: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [1]:
import tensorflow as tf

from imagenet12_util import ret_imagnet12_tf_ds

ds = ret_imagnet12_tf_ds((224, 224))

model = tf.keras.applications.ResNet50()

y_preds = model.predict(ds)

2024-08-16 17:22:38.232486: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-16 17:22:38.255174: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-16 17:22:38.255195: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-16 17:22:38.255213: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-16 17:22:38.259681: I tensorflow/core/platform/cpu_feature_g

Found 50000 files belonging to 1000 classes.


2024-08-16 17:22:40.304753: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


  75/1563 [>.............................] - ETA: 12:13

KeyboardInterrupt: 